# SQL QUERY

In [1]:
# WITH TransfersPerDay AS (
#   SELECT
#     from_address,
#     token_address,
#     block_timestamp
#   FROM
#    bigquery-public-data.crypto_ethereum.token_transfers
#   WHERE
#     block_timestamp >= '2024-06-01 00:00:00 UTC'
#   AND from_address NOT IN (
#     SELECT address
#     FROM
#       bigquery-public-data.crypto_ethereum.contracts
#   )
# ),
# TransactionsPerAddressToken AS (
#   SELECT
#     from_address,
#     token_address,
#     ARRAY_AGG(DISTINCT block_timestamp ORDER BY block_timestamp ASC) AS block_timestamps
#   FROM
#   TransfersPerDay
#   GROUP BY
#     from_address,
#     token_address
# )
# SELECT
#   *
# FROM
#   TransactionsPerAddressToken
# ORDER BY
#   from_address, token_address ;

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import feature_extraction
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [6]:
# df = pd.read_csv("/content/drive/MyDrive/Fichiers/2.Scolarité/1. Jedha_Data_Science/PROJETS/DEMO_DAY/Dataset_src_02/2023-01-15___2023_02-01.csv")
df = pd.read_csv(r"G:\Mon Drive\Fichiers\2.Scolarité\1. Jedha_Data_Science\PROJETS\DEMO_DAY\Dataset_src_02\2023-01-15___2023_02-01.csv")

In [7]:
df

,timestamp,wallet_address,token_address,balance
0,2023-01-19,0xeec9a835df1298587348b5c01048aac2277f340a,0xfe5a28f19934851695783a0c8ccb25d678bb05d3,3.993000e-11
1,2023-01-19,0xeea18c07617cedac2762361231bb908e8b05b17f,0x99a9b7c1116f9ceeb1652de04d5969cce509b069,5.129000e-05
2,2023-01-19,0x3b3fc1f418bef741a954403e5c6cce50df4349e9,0x99a9b7c1116f9ceeb1652de04d5969cce509b069,1.501998e-05
3,2023-01-19,0xb5bbffd7d19e0ed0e4e29f86fb6c70a8379666d4,0x99a9b7c1116f9ceeb1652de04d5969cce509b069,-1.037005e-05
4,2023-01-19,0x1ea014d19b2be0411c996842486420250fb7c9aa,0x99a9b7c1116f9ceeb1652de04d5969cce509b069,-1.073201e-04
...,...,...,...,...
6700654,2023-01-23,0xef843bea95ec79e73b2493770b56fe96139d9b57,0x5ee84583f67d5ecea5420dbb42b462896e7f8d06,-9.597521e+02
6700655,2023-01-23,0x56d181d06d80592da8be947fb0ddd40a816e4fdc,0x5ee84583f67d5ecea5420dbb42b462896e7f8d06,-1.054467e+03
6700656,2023-01-23,0xaf162bf5c9a8413a8628e630f961273d7f3270ab,0x5ee84583f67d5ecea5420dbb42b462896e7f8d06,-1.047172e+03
6700657,2023-01-23,0x7a84078b1813251430e92051cbb615a313378888,0x5ee84583f67d5ecea5420dbb42b462896e7f8d06,-1.044350e+03


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6700659 entries, 0 to 6700658
Data columns (total 4 columns):
 #   Column          Dtype  
---  ------          -----  
 0   timestamp       object 
 1   wallet_address  object 
 2   token_address   object 
 3   balance         float64
dtypes: float64(1), object(3)
memory usage: 204.5+ MB


In [9]:
df["token_address"].value_counts()

token_address
0xdac17f958d2ee523a2206206994597c13d831ec7    1077505
0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48     498756
0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2     201486
0x514910771af9ca656af840dff83e8264ecf986ca     114706
0xdc09ef6ba2a135b350d3d05c409497485d8b0a00     112453
                                               ...   
0xef53b5ba2afb19a8d89ce93b8122d43145932c2a          1
0x805f78d21804bb519a6f2a3d349bed98f9de4fef          1
0xfeabf506f8fcd147d3ed65cc81b82ef87d85df96          1
0xdbd0ff2954abc725336a1b60f6c0a2341703bbdf          1
0x0968c90198f08b67365840fa37631b29fe2aa9fc          1
Name: count, Length: 56610, dtype: int64

In [10]:
print(df["token_address"].nunique())
print((df["token_address"].value_counts()>=100).sum())

56610
5336


In [11]:
print(df["wallet_address"].nunique())
print((df["wallet_address"].value_counts()>=100).sum())


2328444
1364


In [12]:
print(df.isnull().sum())

timestamp         0
wallet_address    0
token_address     0
balance           0
dtype: int64


# PREPROCESS_v1_feature_eng

In [13]:
dataset = df.copy()

In [14]:
# dataset = dataset.sample(n=500000)

In [15]:
dataset['wallet_address'] = dataset['wallet_address'].astype(str)
dataset['token_address'] = dataset['token_address'].astype(str)
dataset["day"] = pd.to_datetime(dataset['timestamp'],format='%Y-%m-%d')
dataset['month'] = pd.to_datetime(dataset['timestamp']).dt.month
dataset['day_of_week'] = pd.to_datetime(dataset['timestamp']).dt.dayofweek

In [16]:
dataset

,timestamp,wallet_address,token_address,balance,day,month,day_of_week
0,2023-01-19,0xeec9a835df1298587348b5c01048aac2277f340a,0xfe5a28f19934851695783a0c8ccb25d678bb05d3,3.993000e-11,2023-01-19,1,3
1,2023-01-19,0xeea18c07617cedac2762361231bb908e8b05b17f,0x99a9b7c1116f9ceeb1652de04d5969cce509b069,5.129000e-05,2023-01-19,1,3
2,2023-01-19,0x3b3fc1f418bef741a954403e5c6cce50df4349e9,0x99a9b7c1116f9ceeb1652de04d5969cce509b069,1.501998e-05,2023-01-19,1,3
3,2023-01-19,0xb5bbffd7d19e0ed0e4e29f86fb6c70a8379666d4,0x99a9b7c1116f9ceeb1652de04d5969cce509b069,-1.037005e-05,2023-01-19,1,3
4,2023-01-19,0x1ea014d19b2be0411c996842486420250fb7c9aa,0x99a9b7c1116f9ceeb1652de04d5969cce509b069,-1.073201e-04,2023-01-19,1,3
...,...,...,...,...,...,...,...
6700654,2023-01-23,0xef843bea95ec79e73b2493770b56fe96139d9b57,0x5ee84583f67d5ecea5420dbb42b462896e7f8d06,-9.597521e+02,2023-01-23,1,0
6700655,2023-01-23,0x56d181d06d80592da8be947fb0ddd40a816e4fdc,0x5ee84583f67d5ecea5420dbb42b462896e7f8d06,-1.054467e+03,2023-01-23,1,0
6700656,2023-01-23,0xaf162bf5c9a8413a8628e630f961273d7f3270ab,0x5ee84583f67d5ecea5420dbb42b462896e7f8d06,-1.047172e+03,2023-01-23,1,0
6700657,2023-01-23,0x7a84078b1813251430e92051cbb615a313378888,0x5ee84583f67d5ecea5420dbb42b462896e7f8d06,-1.044350e+03,2023-01-23,1,0


In [17]:
dataset = dataset.drop(columns = "timestamp")

### Total_transaction

In [18]:
total_transaction_df = dataset.groupby('wallet_address').size().to_frame(name='total_wallet_transactions')
# total_transaction_df = total_transaction_df.reset_index()
print(total_transaction_df.index.nunique())
print(total_transaction_df["total_wallet_transactions"].nunique())
print(total_transaction_df)

2328444
431
                                            total_wallet_transactions
wallet_address                                                       
0x0000000000000000000000000000000000000000                      82117
0x0000000000000000000000000000000000000001                        115
0x0000000000000000000000000000000000000002                          8
0x0000000000000000000000000000000000000003                          5
0x0000000000000000000000000000000000000004                          5
...                                                               ...
0xffffff46eeb8b840e2254d1fa9b1de6a17ce88ff                          4
0xffffff5800b709071d4adc74759ae4b89bef2a9d                          1
0xffffffff5c1f4a12c4552073e8d90f3a391bf7cc                          2
0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c                         20
0xffffffffffffffffffffffffffffffffffffffff                          1

[2328444 rows x 1 columns]


In [19]:
total_transaction_df['total_wallet_transactions'].sort_values(ascending=False)

wallet_address
0x0000000000000000000000000000000000000000    82117
0x0d0707963952f2fba59dd06f2b425ace40b492fe     7501
0xf16e9b0d03470827a95cdfd0cb8a8a3b46969b91     6792
0x75e89d5979e4f6fba9f97c104c2f0afb3f1dcb88     6785
0x000000000000000000000000000000000000dead     5932
                                              ...  
0x848b60ed871bbaa3a9f731263e42b736960f725c        1
0x239b324b275883bac3886778695bdd3ccf84d6f0        1
0x239b385d13f0f2ba30698abf06b2599d92ef9074        1
0x848b4b904a8a9876d6378513e2502201242fb66e        1
0xffffffffffffffffffffffffffffffffffffffff        1
Name: total_wallet_transactions, Length: 2328444, dtype: int64

In [20]:
total_transaction_df.head()

,total_wallet_transactions
wallet_address,
0x0000000000000000000000000000000000000000,82117
0x0000000000000000000000000000000000000001,115
0x0000000000000000000000000000000000000002,8
0x0000000000000000000000000000000000000003,5
0x0000000000000000000000000000000000000004,5


In [21]:
# total_transaction_df['log_total_wallet_transactions'] = np.log1p(total_transaction_df['total_wallet_transactions'])

### Token_most_traded

In [22]:
token_transactions = dataset.groupby('token_address').size().to_frame(name='total_token_transactions')
# token_transactions_df = token_transactions.reset_index()
print(token_transactions.index.nunique())

56610


In [23]:
token_transactions = token_transactions.sort_values("total_token_transactions", ascending=False)
token_transactions

,total_token_transactions
token_address,
0xdac17f958d2ee523a2206206994597c13d831ec7,1077505
0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,498756
0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,201486
0x514910771af9ca656af840dff83e8264ecf986ca,114706
0xdc09ef6ba2a135b350d3d05c409497485d8b0a00,112453
...,...
0x35be4379fa51d3748585532915142b5bf64534c8,1
0xc9da1f7e3fb6422028428cb2314aaa7836ac8afa,1
0x35cd5afde3994608934f3bec6621ba2f0606985f,1


In [24]:
print((dataset["token_address"].value_counts()>=10000).sum())

49


In [25]:
top_tokens = token_transactions.head(50)
top_tokens = top_tokens.index.tolist()
top_tokens

['0xdac17f958d2ee523a2206206994597c13d831ec7',
 '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
 '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
 '0x514910771af9ca656af840dff83e8264ecf986ca',
 '0xdc09ef6ba2a135b350d3d05c409497485d8b0a00',
 '0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce',
 '0x4d224452801aced8b2f0aebe155379bb5d594381',
 '0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0',
 '0x6b175474e89094c44da98b954eedeac495271d0f',
 '0x0000000000a39bb272e79075ade125fd351887ac',
 '0x06450dee7fd2fb8e39061434babcfc05599a6fb8',
 '0x15d4c048f83bd7e37d49ea4c83a07267ec4203da',
 '0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85',
 '0x764aeebcf425d56800ef2c84f2578689415a2daa',
 '0x4fabb145d64652a948d72533023f6e7a623c7c53',
 '0x77372a4cc66063575b05b44481f059be356964a4',
 '0x4a220e6096b25eadb88358cb44068a3248254675',
 '0x9813037ee2218799597d83d4a5b6f3b6778218d9',
 '0x05237e2bd2dfab39a135d254cabae94b183c8bad',
 '0x2b591e99afe9f32eaa6214f7b7629768c40eeb39',
 '0x3845badade8e6dff049820680d1f14bd3903a5d0',
 '0x0f5d2fb29

In [26]:
# main_tokens = ["0xdac17f958d2ee523a2206206994597c13d831ec7",
# "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48",
# "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2",
# "0x514910771af9ca656af840dff83e8264ecf986ca",
# "0xdc09ef6ba2a135b350d3d05c409497485d8b0a00"]

data_kmeans_2 = dataset[dataset["token_address"].isin(top_tokens)]

In [27]:
print(data_kmeans_2.shape)
print(data_kmeans_2["wallet_address"].nunique())
data_kmeans_2.head()

(3077292, 6)
1558736


,wallet_address,token_address,balance,day,month,day_of_week
116,0x8486f15ab5a8fc33ab503fd296da82d15ee2c837,0xd26114cd6ee289accf82350c8d8487fedb8a0c07,-8.888511e+08,2023-01-19,1,3
117,0x60c161480399efe8386540d39d270768055614f4,0xd26114cd6ee289accf82350c8d8487fedb8a0c07,2.327733e+08,2023-01-19,1,3
126,0xa27502a0bfdf8cc5d1ac080b55885541d14d190d,0x7eba6418388cac0b81c26f9b76d7ea6877bf8308,2.381356e-08,2023-01-19,1,3
127,0x8d911d8919eb1f4c3ba4495c198ea88487ac6441,0x7eba6418388cac0b81c26f9b76d7ea6877bf8308,2.599040e-08,2023-01-19,1,3
128,0x9c74a256452a09312d2ba4b2bf42f23c6b998117,0x7eba6418388cac0b81c26f9b76d7ea6877bf8308,3.479445e-08,2023-01-19,1,3


In [28]:
data_pivot_table = data_kmeans_2.pivot_table(index='wallet_address', columns='token_address', aggfunc='size')

In [29]:
print(data_pivot_table.shape)
data_pivot_table.head(10)

(1558736, 50)


token_address,0x0000000000a39bb272e79075ade125fd351887ac,0x05237e2bd2dfab39a135d254cabae94b183c8bad,0x06450dee7fd2fb8e39061434babcfc05599a6fb8,0x0f5d2fb29fb7d3cfee444a200298f468908cc942,0x1562e3428ed1d6cdbdb1695cc505aba33ab4ea5e,0x15d4c048f83bd7e37d49ea4c83a07267ec4203da,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,0x340f8999e1f45a7155e4e2be15e20fc64c2d2d13,...,0xc0f6cc4b7e67f0518851068601c9f7c4ef18b201,0xc944e90c64b2c07662a292be6244bdf05cda44a7,0xd26114cd6ee289accf82350c8d8487fedb8a0c07,0xd533a949740bb3306d119cc777fa900ba034cd52,0xdac17f958d2ee523a2206206994597c13d831ec7,0xdc09ef6ba2a135b350d3d05c409497485d8b0a00,0xde30da39c46104798bb5aa3fe8b9e0e1f348163f,0xe1b81cd6a494cbca06a8e2055a62c2cf0fa5a8ac,0xf411903cbc70a74d22900a5de66a2dda66507255,0xfa969c60a78195c631787d4585ba15a07578c979
wallet_address,,,,,,,,,,,,,,,,,,,,,
0x0000000000000000000000000000000000000000,17.0,NaN,17.0,NaN,1.0,17.0,NaN,7.0,17.0,1.0,...,1.0,17.0,NaN,17.0,4.0,NaN,NaN,6.0,17.0,1.0
0x0000000000000000000000000000000000000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0x0000000000000000000000000000000000001010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0x0000000000000000000000000000000000002ca0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0x000000000000000000000000000000000000dead,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,1.0
0x0000000000000000000000000000000000abcdef,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0x000000000000000000000000000000006303220b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0x00000000000000000000000000007e1787bae39b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0x00000000000057667bfefe3158822e6fb4058a6d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
data_pivot_table_sc = pd.concat([data_pivot_table, total_transaction_df], axis = 1)
data_pivot_table_sc.shape

(2328444, 51)

In [52]:
data_pivot_table_sc = data_pivot_table_sc.fillna(0)

In [53]:
# Wallet avec 0 partout dans le dataframe => Wallet qui n'ont tradé aucun des 50 pricnipaux tokens

In [54]:
# Pre-processing par proportion de transaction vs total transaction wallet.
# data_pivot_table_sc.iloc[:, :-1] = data_pivot_table_sc.iloc[:, :-1].div(data_pivot_table_sc['total_wallet_transactions'], axis=0)

In [55]:
data_pivot_table_sc.head()

,0x0000000000a39bb272e79075ade125fd351887ac,0x05237e2bd2dfab39a135d254cabae94b183c8bad,0x06450dee7fd2fb8e39061434babcfc05599a6fb8,0x0f5d2fb29fb7d3cfee444a200298f468908cc942,0x1562e3428ed1d6cdbdb1695cc505aba33ab4ea5e,0x15d4c048f83bd7e37d49ea4c83a07267ec4203da,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,0x340f8999e1f45a7155e4e2be15e20fc64c2d2d13,...,0xc944e90c64b2c07662a292be6244bdf05cda44a7,0xd26114cd6ee289accf82350c8d8487fedb8a0c07,0xd533a949740bb3306d119cc777fa900ba034cd52,0xdac17f958d2ee523a2206206994597c13d831ec7,0xdc09ef6ba2a135b350d3d05c409497485d8b0a00,0xde30da39c46104798bb5aa3fe8b9e0e1f348163f,0xe1b81cd6a494cbca06a8e2055a62c2cf0fa5a8ac,0xf411903cbc70a74d22900a5de66a2dda66507255,0xfa969c60a78195c631787d4585ba15a07578c979,total_wallet_transactions
wallet_address,,,,,,,,,,,,,,,,,,,,,
0x0000000000000000000000000000000000000000,17.0,0.0,17.0,0.0,1.0,17.0,0.0,7.0,17.0,1.0,...,17.0,0.0,17.0,4.0,0.0,0.0,6.0,17.0,1.0,82117
0x0000000000000000000000000000000000000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,115
0x0000000000000000000000000000000000001010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
0x0000000000000000000000000000000000002ca0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
0x000000000000000000000000000000000000dead,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,1.0,5932


########################Model_Training#####################

## K-Means_Test_2_pivot_table_main_tokens

In [56]:
scaler = StandardScaler()
data_pivot_table_sc[data_pivot_table_sc.columns] = scaler.fit_transform(data_pivot_table_sc[data_pivot_table_sc.columns])
data_pivot_table_sc

array([[ 4.86486344e+01, -4.64451827e-02,  7.83814741e+01, ...,
         1.07334218e+02,  6.34228772e+00,  1.47930614e+03],
       [-6.50561142e-02, -4.64451827e-02, -1.00445077e-01, ...,
        -5.61673538e-02, -4.88964107e-02,  2.01991011e+00],
       [-6.50561142e-02, -4.64451827e-02, -1.00445077e-01, ...,
        -5.61673538e-02, -4.88964107e-02, -3.38279684e-02],
       ...,
       [-6.50561142e-02, -4.64451827e-02, -1.00445077e-01, ...,
        -5.61673538e-02, -4.88964107e-02,  2.02177705e-02],
       [-6.50561142e-02, -4.64451827e-02, -1.00445077e-01, ...,
        -5.61673538e-02, -4.88964107e-02, -3.38279684e-02],
       [-6.50561142e-02, -4.64451827e-02, -1.00445077e-01, ...,
        -5.61673538e-02, -4.88964107e-02, -3.38279684e-02]])

In [36]:
# wcss =  []
# k = []

# for i in range (2,12):
#     kmeans = KMeans(n_clusters= i, random_state = 0, n_init = 10)
#     kmeans.fit(data_pivot_table_sc)
#     wcss.append(kmeans.inertia_)
#     k.append(i)
#     print("WCSS for K={} --> {}".format(i, wcss[-1]))

In [37]:
# plt.figure(figsize=(10, 6))
# plt.plot(k, wcss, 'bo-', markersize=8, linewidth=2)
# plt.xlabel('Number of Clusters (k)')
# plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
# plt.title('Elbow Method for data_standard_scaled')
# plt.xticks(range(1, 20))
# plt.grid(True)
# plt.show()

In [57]:
kmeans = KMeans(n_clusters=10, random_state=42)

In [58]:
kmeans.fit(data_pivot_table_sc)

c:\Users\PC\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=10, random_state=42)

In [40]:
data_for_kmeans.shape

(2328444, 51)

In [59]:
kmeans.labels_

array([4, 0, 0, ..., 0, 0, 0])

In [61]:
data_pivot_table_sc['cluster_label_kmeans'] = kmeans.labels_
data_pivot_table_sc["cluster_label_kmeans"].value_counts()
data_pivot_table_sc.head()

,0x0000000000a39bb272e79075ade125fd351887ac,0x05237e2bd2dfab39a135d254cabae94b183c8bad,0x06450dee7fd2fb8e39061434babcfc05599a6fb8,0x0f5d2fb29fb7d3cfee444a200298f468908cc942,0x1562e3428ed1d6cdbdb1695cc505aba33ab4ea5e,0x15d4c048f83bd7e37d49ea4c83a07267ec4203da,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,0x340f8999e1f45a7155e4e2be15e20fc64c2d2d13,...,0xd26114cd6ee289accf82350c8d8487fedb8a0c07,0xd533a949740bb3306d119cc777fa900ba034cd52,0xdac17f958d2ee523a2206206994597c13d831ec7,0xdc09ef6ba2a135b350d3d05c409497485d8b0a00,0xde30da39c46104798bb5aa3fe8b9e0e1f348163f,0xe1b81cd6a494cbca06a8e2055a62c2cf0fa5a8ac,0xf411903cbc70a74d22900a5de66a2dda66507255,0xfa969c60a78195c631787d4585ba15a07578c979,total_wallet_transactions,cluster_label_kmeans
wallet_address,,,,,,,,,,,,,,,,,,,,,
0x0000000000000000000000000000000000000000,17.0,0.0,17.0,0.0,1.0,17.0,0.0,7.0,17.0,1.0,...,0.0,17.0,4.0,0.0,0.0,6.0,17.0,1.0,82117,4
0x0000000000000000000000000000000000000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,115,0
0x0000000000000000000000000000000000001010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
0x0000000000000000000000000000000000002ca0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
0x000000000000000000000000000000000000dead,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,1.0,5932,1


In [ ]:
# plt.hist(data_pivot_table_sc["cluster_label_kmeans"])
# plt.xticks(range(data_pivot_table_sc["cluster_label_kmeans"].max() + 1))
# plt.xlabel("Cluster Label")
# plt.ylabel("Number of Data Points")
# plt.title("Distribution of Data Points in Clusters (Sample Dataset)")
# plt.show()

In [62]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
# Reduce dimensions using PCA for visualization

pca = PCA(n_components=3)
pca_components = pca.fit_transform(data_pivot_table_sc)

In [63]:
pca_df = pd.DataFrame(data=pca_components, columns=['PCA1', 'PCA2', 'PCA3'])
pca_df['Cluster'] = kmeans.labels_

In [65]:
figpx3d = px.scatter_3d(pca_df[:5000], x = 'PCA1', y = 'PCA2', z = 'PCA3', color= 'Cluster')
figpx3d.show()

In [ ]:
# Create a DataFrame with PCA components and cluster labels
pca_df = pd.DataFrame(data=pca_components, columns=['PCA1', 'PCA2', 'PCA3'])
pca_df['Cluster'] = kmeans.labels_


# Create a figure and 3D axes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot (without coloring by cluster for this example)
ax.scatter(pca_df['PCA1'], pca_df['PCA2'], pca_df['PCA3'], c = pca_df['Cluster']) 

# Labels and title
ax.set_xlabel('PCA1')
ax.set_ylabel('PCA2')
ax.set_zlabel('PCA3')
ax.set_title('3D PCA Space')
plt.show()

## Birch_Model

In [68]:
from sklearn.cluster import Birch
birch_model = Birch(n_clusters=2)  # Use n_clusters=None to let Birch determine the number of clusters
birch_model.fit(data_pivot_table_sc)

birch_labels = birch_model.predict(data_pivot_table_sc)

# Optionally, you can assign the labels back to the DataFrame
data_pivot_table_sc['Cluster_labels_birch'] = birch_labels

MemoryError: Unable to allocate 28.5 GiB for an array with shape (3828168750,) and data type float64

In [ ]:
tsne = TSNE(n_components=2, random_state=42)
data_tsne = tsne.fit_transform(data_pivot_table_sc)

data_tsne_df = pd.DataFrame(data=data_tsne, columns=['TSNE1', 'TSNE2'])
data_tsne_df['Cluster'] = data_pivot_table_sc['Cluster']

plt.figure(figsize=(10, 6))
sns.scatterplot(x='TSNE1', y='TSNE2', hue='Cluster', palette='viridis', data=data_tsne_df, legend='full')
plt.title('Birch Clustering Results (t-SNE)')
plt.show()